# understand the basic of applying gpt4o vision
https://cookbook.openai.com/examples/gpt_with_vision_for_video_understanding

another good reference to adjust the parameters of processing video
https://platform.openai.com/docs/guides/vision

In [1]:
from IPython.display import display, Image, Audio

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
from openai import OpenAI
import os
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [2]:
video = cv2.VideoCapture("output_video.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")

3870 frames read.


In [6]:
# Assuming base64Frames is already populated with base64-encoded frames
base64Frames_first2 = base64Frames[:2]  # Slice the first two elements

# Print the result to verify
print("First two frames in base64Frames_first2:")
for i, frame in enumerate(base64Frames_first2, start=1):
    print(f"Frame {i}: {frame[:50]}...")  # Print the first 50 characters for brevity

First two frames in base64Frames_first2:
Frame 1: /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAg...
Frame 2: /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAg...


In [7]:
'''
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
'''
# Assuming base64Frames_first2 is already defined and contains the first two frames
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames_first2),
        ],
    },
]

params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

Join me on a scenic drive where technology meets the open road. This video takes you on a journey through a picturesque highway lined with construction cones and framed by a vibrant mix of cloudy skies and woodland. Experience the seamless integration of in-car navigation systems, capturing the essence of modern driving amidst the ever-changing landscape. Perfect for those who appreciate the blend of innovation and nature. Buckle up and enjoy the ride!
